<a href="https://colab.research.google.com/github/SarathSabu/Python-Notebooks/blob/main/Hospital_data_efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hospital Data DEA


Install amplpy, pandas and other packages

In [ ]:
!pip install -q amplpy ampltools pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.6 MB/s eta 0:00:00


Setup AMPL and select solver model(s)

In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


Setup Google Drive. Data files can then be accessed from MyDrive on Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import pandas and use it to read data from file.
Define data.

In [ ]:
import pandas as pd

hospital_data = pd.read_excel("/content/drive/MyDrive/info 645/hospital_data.xlsx",
                          sheet_name="Sheet1",
                          index_col=0)

col_names = list(hospital_data)
I = [i for i in col_names if "Input" in i]
J = [j for j in col_names if "Output" in j]
H = list(hospital_data.index)
print(hospital_data)

input_data = hospital_data[I].stack().to_dict()
output_data = hospital_data[J].stack().to_dict()

print(input_data)
print(output_data)

           Input 1  Input 2   Input 3  Output 1  Output 2  Output 3  Output 4  \
Hospital                                                                        
1          83.7033   2.9940    8.0480   51.1574   32.7153   66.2448   35.7387   
2          91.5586   1.1948   77.4837    9.8605   70.3838  127.5586   59.4214   
3          43.7093   0.8526  185.2281    8.7410   31.4517    2.2810   31.1873   
4          79.9493   1.1131   63.5319   19.5161    0.5712   82.8274  105.6284   
5          20.2349   0.0052   10.0501    1.2792    8.8982    1.3348    7.9021   
...            ...      ...       ...       ...       ...       ...       ...   
346        16.0298   0.1653    5.1617    1.5381   13.1262   19.3608   20.1754   
347       300.6545   1.4434  192.6605   27.9351  154.2200  138.1157   66.2033   
348       311.0615   0.4331  124.7328   32.5675  234.1202   79.1754  108.3292   
349       357.7746   3.4435  145.9937   12.4487   55.3048   25.9118  517.2526   
350        22.5251   0.1116 


Define model.

In [ ]:
ampl.eval ('''

reset;
set I;
set J;
set H;

param input_data {h in H, i in I} ;
param output_data {h in H, j in J} ;
param k symbolic; #The hospitals we are checking for efficiency

var x {i in I} >= 0;
var y {j in J} >= 0;

maximize total_value_objective: sum {j in J} output_data[k,j] * y[j] ;

subject to
  efficiency_constraint {h in H} : sum {j in J} output_data[h,j]*y[j] <=
                                   sum {i in I} input_data[h,i]*x[i] ;
  normalization_constraint : sum {i in I} input_data[k,i]*x[i] == 1 ;

''')

Provide data to the model.

In [ ]:
ampl.set['I'] = I
ampl.set['J'] = J
ampl.set['H'] = H

ampl.param['input_data'] = input_data
ampl.param['output_data'] = output_data
ampl.param['k'] = 1

Display problem formulation.


In [ ]:
ampl.eval('''expand;''')

maximize total_value_objective:
	51.1574*y['Output 1'] + 32.7153*y['Output 2'] + 66.2448*y['Output 3']
	 + 35.7387*y['Output 4'] + 49.023*y['Output 5'] + 49.2987*y['Output 6']
	 + 39.5518*y['Output 7'];

subject to efficiency_constraint[1]:
	-83.7033*x['Input 1'] - 2.994*x['Input 2'] - 8.048*x['Input 3'] + 
	51.1574*y['Output 1'] + 32.7153*y['Output 2'] + 66.2448*y['Output 3']
	 + 35.7387*y['Output 4'] + 49.023*y['Output 5'] + 49.2987*y['Output 6']
	 + 39.5518*y['Output 7'] <= 0;

subject to efficiency_constraint[2]:
	-91.5586*x['Input 1'] - 1.1948*x['Input 2'] - 77.4837*x['Input 3'] + 
	9.8605*y['Output 1'] + 70.3838*y['Output 2'] + 127.559*y['Output 3'] + 
	59.4214*y['Output 4'] + 88.7819*y['Output 5'] + 46.5052*y['Output 6']
	 + 2.3613*y['Output 7'] <= 0;

subject to efficiency_constraint[3]:
	-43.7093*x['Input 1'] - 0.8526*x['Input 2'] - 185.228*x['Input 3'] + 
	8.741*y['Output 1'] + 31.4517*y['Output 2'] + 2.281*y['Output 3'] + 
	31.1873*y['Output 4'] + 30.2933*y['Output 5'] + 18.

Set solver and solve.

In [ ]:
ampl.setOption('solver', 'cbc')
ampl.setOption('solver_msg','0')

result_count = 0
for k in H:
  ampl.param['k'] = k
  ampl.solve()
  obj = ampl.getObjective('total_value_objective')
  if obj.get().value() > 0.999:
    print("%s %f" % (k,obj.get().value()))
    result_count += 1
print("Number of results: %d" % result_count)



cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: 9 1.000000
cbc 2.10.10: cbc 2.10.10: 11 1.000000
cbc 2.10.10: 12 1.000000
cbc 2.10.10: cbc 2.10.10: 14 1.000000
cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: 

In [ ]:
# Print results
obj = ampl.getObjective('total_value_objective')
print("\n")
print("Total value is: ", obj.get().value(), "\n")
ampl.display('x');
ampl.display('y');



Total value is:  0.3152673986832677 

x [*] :=
'Input 1'  0.0145646
'Input 2'  3.04484
'Input 3'  0.0203786
;

y [*] :=
'Output 1'  0.0158663
'Output 2'  0
'Output 3'  0
'Output 4'  0
'Output 5'  0
'Output 6'  0
'Output 7'  0.0568055
;

